In [ ]:
# 一次微调
# 基座模型：LLaMA3-8B-Chat，先采用 alpaca_gpt4_zh 数据集对 LLaMA3-8B-Chat 进行一次微调，提高中文对话能力

# 配置: /mnt/workspace/Mindpilot/LLaMA-Factory/examples/train_lora/llama3_lora_sft.yaml
# dataset: /mnt/workspace/Mindpilot/dataset/alpaca_gpt4_data_zh
# 参数保存路径: /mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/sft_alpaca_v1

CUDA_VISIBLE_DEVICES=0 llamafactory-cli train /mnt/workspace/Mindpilot/LLaMA-Factory/examples/train_lora/llama3_lora_sft.yaml

In [ ]:
# 一次微调模型导出

# 模型路径: /mnt/workspace/Mindpilot/Meta-Llama-3-8B-Instruct
# 参数保存路径: /mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/sft_alpaca_v1
# 导出配置路径：/mnt/workspace/Mindpilot/LLaMA-Factory/examples/merge_lora/llama3_lora_sft.yaml
# export_dir: /mnt/workspace/Mindpilot/output/Meta-Llama-3-8B-Instruct-alpaca_gpt4_zh-sfted_v1

CUDA_VISIBLE_DEVICES=0 llamafactory-cli export /mnt/workspace/Mindpilot/LLaMA-Factory/examples/merge_lora/llama3_lora_sft.yaml

In [ ]:
# 一次微调效果

# CUDA_VISIBLE_DEVICES=0 llamafactory-cli chat /mnt/workspace/Mindpilot/LLaMA-Factory/examples/inference/llama3_lora_sft.yaml
CUDA_VISIBLE_DEVICES=0 GRADIO_SHARE=1 llamafactory-cli webui

In [ ]:
# 二次微调
# 基座模型：Meta-Llama-3-8B-Instruct-alpaca_gpt4_zh-sfted_v1，采用 PsyQA 数据集对一次微调后的模型进行sft微调

# 先将 psyQA_full_origin.json 数据集转换为适配的格式
python /mnt/workspace/Mindpilot/dataset/data_process.py

# 修改配置文件
# 配置: /mnt/workspace/Mindpilot/LLaMA-Factory/examples/train_lora/llama3_lora_sft.yaml
# 模型路径: /mnt/workspace/Mindpilot/output/Meta-Llama-3-8B-Instruct-alpaca_gpt4_zh-sfted_v1 
# dataset: /mnt/workspace/Mindpilot/dataset/PsyQA
# 参数保存路径：/mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/sft_psyQA_v2

CUDA_VISIBLE_DEVICES=0 llamafactory-cli train /mnt/workspace/Mindpilot/LLaMA-Factory/examples/train_lora/llama3_lora_sft.yaml

In [ ]:
# 二次微调模型导出

# 模型路径：/mnt/workspace/Mindpilot/output/Meta-Llama-3-8B-Instruct-alpaca_gpt4_zh-sfted_v1 
# 参数保存路径：/mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/sft_psyQA_v2
# 导出配置路径：/mnt/workspace/Mindpilot/LLaMA-Factory/examples/merge_lora/llama3_lora_sft.yaml
# export_dir: /mnt/workspace/Mindpilot/output/Meta-Llama-3-8B-Instruct-psyQA-sfted_v2

CUDA_VISIBLE_DEVICES=0 llamafactory-cli export /mnt/workspace/Mindpilot/LLaMA-Factory/examples/merge_lora/llama3_lora_sft.yaml

In [ ]:
# 二次微调效果

# CUDA_VISIBLE_DEVICES=0 GRADIO_SHARE=1 llamafactory-cli webui
CUDA_VISIBLE_DEVICES=0 llamafactory-cli chat /mnt/workspace/Mindpilot/LLaMA-Factory/examples/inference/llama3_lora_sft.yaml

In [ ]:
# 训练奖励模型

# 基座模型：Meta-Llama-3-8B-Instruct-psyQA-sfted_v2，以二次微调的模型作为基座
# 数据集（已经加入偏好标注）：PsyQA_comparison.json
# 配置：/mnt/workspace/Mindpilot/LLaMA-Factory/examples/train_lora/llama3_lora_reward.yaml
# 模型路径：/mnt/workspace/Mindpilot/output/Meta-Llama-3-8B-Instruct-psyQA-sfted_v2
# dataset: /mnt/workspace/Mindpilot/dataset/PsyQA_comparison
# 参数保存路径：/mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/rm

CUDA_VISIBLE_DEVICES=0 llamafactory-cli train /mnt/workspace/Mindpilot/LLaMA-Factory/examples/train_lora/llama3_lora_reward.yaml

In [ ]:
# PPO算法执行强化学习

# 以二次微调的模型作为基座（Meta-Llama-3-8B-Instruct-psyQA-sfted_v2），加入奖励模型训练的参数（/mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/rm）
# 数据集采用二次微调时的PsyQA.json

# 强化学习配置：/mnt/workspace/Mindpilot/LLaMA-Factory/examples/train_lora/llama3_lora_ppo.yaml
# 模型路径：/mnt/workspace/Mindpilot/output/Meta-Llama-3-8B-Instruct-psyQA-sfted_v2
# 奖励模型参数路径：/mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/rm
# dataset：/mnt/workspace/Mindpilot/dataset/PsyQA
# 强化学习参数保存路径：/mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/ppo

CUDA_VISIBLE_DEVICES=0 llamafactory-cli train /mnt/workspace/Mindpilot/LLaMA-Factory/examples/train_lora/llama3_lora_ppo.yaml

In [ ]:
# RLHF模型导出

# 模型路径：/mnt/workspace/Mindpilot/output/Meta-Llama-3-8B-Instruct-psyQA-sfted_v2
# 参数保存路径：/mnt/workspace/Mindpilot/saves/LLaMA3-8B-Chat/lora/ppo
# 导出配置路径：/mnt/workspace/Mindpilot/LLaMA-Factory/examples/merge_lora/llama3_lora_sft.yaml
# export_dir: /mnt/workspace/Mindpilot/output/Meta-Llama-3-8B-Instruct-psyQA-RLHG

CUDA_VISIBLE_DEVICES=0 llamafactory-cli export /mnt/workspace/Mindpilot/LLaMA-Factory/examples/merge_lora/llama3_lora_sft.yaml

In [ ]:
# 推理
CUDA_VISIBLE_DEVICES=0 llamafactory-cli chat /mnt/workspace/Mindpilot/LLaMA-Factory/examples/inference/llama3_lora_sft.yaml